In [2]:
#Import dependencies and create html filepath
from twitter_scraper import get_tweets
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import requests
import pprint

In [3]:
############ NASA MARS NEWS ############
#Set Executable Path & identify url path
executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
#Parse webpage
html= browser.html
soup = bs(html, 'html.parser')

In [5]:
# First card lives in the first 'li "slide" class
#   <ul class="item_list">
#     <li class="slide">
first_article = soup.select_one('ul.item_list li.slide')
news_title = first_article.find('div', class_='content_title').text
print(news_title)

8 Martian Postcards to Celebrate Curiosity's Landing Anniversary


In [6]:
#Paragraph Text for same article
news_p = first_article.find('div', class_='article_teaser_body').text
print(news_p)

The NASA rover touched down eight years ago, on Aug. 5, 2012, and will soon be joined by a second rover, Perseverance.


In [7]:
############ JPL MARS FEATURED IMAGE ############
#Set Executable Path & identify URL path
executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)
iurl= 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(iurl)

In [8]:
#Go to full image button to see featured image via id='full_image'
full_image_button = browser.find_by_id('full_image')
full_image_button.click()

In [9]:
#Go to 'more info' button via element present by text 'more info'
more_info_element = browser.find_link_by_partial_text('more info')
more_info_element.click()

In [10]:
#Grab full-size image url .jpg
html = browser.html
image_souped = bs(html, 'html.parser')
image_url = image_souped.select_one('figure.lede a img').get('src')
image_url

'/spaceimages/images/largesize/PIA16883_hires.jpg'

In [11]:
#Combine main url with image route
direct_image_url = f"https://www.jpl.nasa.gov{image_url}"
print(direct_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16883_hires.jpg


In [12]:
############# MARS TWITTER WEATHER ############
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [13]:
html_weather=browser.html
soup=bs(html_weather, 'html.parser')
tweets=[]
for tweet in get_tweets('@MarsWxReport', pages=1):
    tw=tweet['text']
    tweets.append(tw)

In [14]:
tweets[0]

'InSight sol 598 (2020-08-01) low -91.6ºC (-132.9ºF) high -15.1ºC (4.8ºF)\nwinds from the WNW at 7.1 m/s (15.8 mph) gusting to 19.2 m/s (43.0 mph)\npressure at 7.90 hPa pic.twitter.com/xedt9nLLfq'

In [19]:
first_tweet=tweets[0]

In [15]:
############ MARS FACTS ############
furl = 'https://space-facts.com/mars/'
fact_tables = pd.read_html(furl)
mars_tables = fact_tables[0]
mars_tables.columns = ["Description","Value"]
mars_tables.set_index('Description', inplace=True)
mars_tables.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [16]:
############ MARS HEMISPHERES ############
executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [17]:
hemisphere_image_urls = []

# Get a List of All the Hemispheres
links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    # Find element to navigate to each hemisphere page
    browser.find_by_css("a.product-item h3")[item].click()
    # Get hemisphere name text
    hemisphere["Title"] = browser.find_by_css("h2.title").text
    # Extract href tag from Sample jpg
    sample_element = browser.find_link_by_text("Sample").first
    hemisphere["Image URL"] = sample_element["href"]     
    # Append info dictionary to above list
    hemisphere_image_urls.append(hemisphere)   
    # Return to main Mars Hemisphere page to restart the for loop at the top
    browser.back()

hemisphere_image_urls

[{'Title': 'Cerberus Hemisphere Enhanced',
  'Image URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'Image URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'Image URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'Image URL': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [21]:
scrape_data_dict= {
"News Title": news_title,
"News Paragraph": news_p,
"Image": direct_image_url,
"Tweet":first_tweet,
"Mars Facts": mars_tables,
"Hemisphere Image URL's":hemisphere_image_urls
}

print(scrape_data_dict)

{'News Title': "8 Martian Postcards to Celebrate Curiosity's Landing Anniversary", 'News Paragraph': 'The NASA rover touched down eight years ago, on Aug. 5, 2012, and will soon be joined by a second rover, Perseverance.', 'Image': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16883_hires.jpg', 'Tweet': 'InSight sol 598 (2020-08-01) low -91.6ºC (-132.9ºF) high -15.1ºC (4.8ºF)\nwinds from the WNW at 7.1 m/s (15.8 mph) gusting to 19.2 m/s (43.0 mph)\npressure at 7.90 hPa pic.twitter.com/xedt9nLLfq', 'Mars Facts':                                               Value
Description                                        
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.39 × 10^23 kg (0.11 Earths)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.38 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                   -87 to -5 °C
Fir

In [22]:
def scrape():
    ############ NASA MARS NEWS ############

    #Set Executable Path & identify url path
    executable_path = {"executable_path": "./chromedriver.exe"}
    browser = Browser("chrome", **executable_path, headless=False)
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)

    #Parse webpage
    html= browser.html
    soup = bs(html, 'html.parser')
    first_article = soup.select_one('ul.item_list li.slide')
    news_title = first_article.find('div', class_='content_title').text
    news_p = first_article.find('div', class_='article_teaser_body').text

    ############ JPL MARS FEATURED IMAGE ############
    #Set Executable Path & identify URL path
    executable_path = {"executable_path": "./chromedriver.exe"}
    browser = Browser("chrome", **executable_path, headless=False)
    iurl= 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(iurl)

    full_image_button = browser.find_by_id('full_image')
    full_image_button.click()
    more_info_element = browser.find_link_by_partial_text('more info')
    more_info_element.click()
    html = browser.html
    image_souped = bs(html, 'html.parser')
    image_url = image_souped.select_one('figure.lede a img').get('src')
    direct_image_url = f"https://www.jpl.nasa.gov{image_url}"

    ############# MARS TWITTER WEATHER ############
    weather_url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(weather_url)
    html_weather=browser.html
    soup=bs(html_weather, 'html.parser')
    tweets=[]
    for tweet in get_tweets('@MarsWxReport', pages=1):
        tw=tweet['text']
        tweets.append(tw)
    first_tweet = tweets[0]

    ############ MARS FACTS ############
    furl = 'https://space-facts.com/mars/'
    fact_tables = pd.read_html(furl)
    mars_tables = fact_tables[0]
    mars_tables.columns = ["Description","Value"]
    mars_tables.set_index('Description', inplace=True)
    mars_tables.to_html()

    ############ MARS HEMISPHERES ############
    hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg"},
    {"title": "Cerberus Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg"},
    {"title": "Schiaparelli Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg"},
    {"title": "Syrtis Major Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg"},
    ]

    scrape_data_dict= {
    "News Title": news_title,
    "News Paragraph": news_p,
    "Image": direct_image_url,
    "Tweet":first_tweet,
    "Mars Facts": mars_tables,
    "Hemisphere Image URL's":hemisphere_image_urls
    }

    return scrape_data_dict

In [23]:
scrape()

{'News Title': "8 Martian Postcards to Celebrate Curiosity's Landing Anniversary",
 'News Paragraph': 'The NASA rover touched down eight years ago, on Aug. 5, 2012, and will soon be joined by a second rover, Perseverance.',
 'Image': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16883_hires.jpg',
 'Tweet': 'InSight sol 598 (2020-08-01) low -91.6ºC (-132.9ºF) high -15.1ºC (4.8ºF)\nwinds from the WNW at 7.1 m/s (15.8 mph) gusting to 19.2 m/s (43.0 mph)\npressure at 7.90 hPa pic.twitter.com/xedt9nLLfq',
 'Mars Facts':                                               Value
 Description                                        
 Equatorial Diameter:                       6,792 km
 Polar Diameter:                            6,752 km
 Mass:                 6.39 × 10^23 kg (0.11 Earths)
 Moons:                          2 (Phobos & Deimos)
 Orbit Distance:            227,943,824 km (1.38 AU)
 Orbit Period:                  687 days (1.9 years)
 Surface Temperature:                   -87 